# **Topic Modeling with Quantized LLMs**
*Create easily interpretable topics with BERTopic and LLMs*
<br>

<img src="https://pbs.twimg.com/media/GDA2JQoX0AElWoq?format=jpg&name=large">


---
        
💡 **NOTE**: We will want to use a GPU to run both 7B LLMs as well as BERTopic for this use case. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

We will start by installing a number of packages that we are going to use throughout this example:

# 📄 **Data**

We are going to apply topic modeling on a number of ArXiv abstracts. They are a great source for topic modeling since they contain a wide variety of topics and are generally well-written.

In [3]:
from datasets import load_dataset
from github_search.ir.evaluate_bm25 import load_ir_data, load_generation_metrics_df, RetrievalConfig, get_retriever


columns = ["dependencies", "tasks"]

from zenml.client import Client

artifact = Client().get_artifact('8c9d506c-797d-4429-b9bc-ae0ba519117e')
search_df = artifact.load()

In [9]:
def get_docs(df, doc_columns):
    docs = df[doc_columns[0]]
    for col in doc_columns[1:]:
        docs = docs + " " + df[col]
    return docs.str.replace("/", " ").to_list()

In [11]:
docs = get_docs(search_df, columns)

# 🦙 **Quantized LLM**

To use a quantized LLMs, we make use of the `GGUF` format together with `llama-cpp-python`. When you access any of [TheBloke's quantized models](https://huggingface.co/TheBloke), you can click on files and find specific quantized formats. We are going with a 4-bit quantized model:

In [12]:
#!wget https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q4_K_M.gguf
# !wget https://huggingface.co/TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/resolve/main/dolphin-2.7-mixtral-8x7b.Q3_K_M.gguf

Next, we load the model using `llama-cpp-python`:

from llama_cpp import Llama

# Use llama.cpp to load in a Quantized LLM
llm = Llama(model_path="openhermes-2.5-mistral-7b.Q4_K_M.gguf", n_gpu_layers=-1, n_ctx=4096, stop=["Q:", "\n"])

We pass the LLM to BERTopic with a custom prompt. Let's add `KeyBERTInspired` representations to compare the LLM representations with:

In [13]:
from bertopic.representation import KeyBERTInspired, LlamaCPP

prompt = """ Q:
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the above information, can you give a short label of the topic of at most 5 words?
A:
"""

representation_model = {
    "KeyBERT": KeyBERTInspired(),
    #"LLM": LlamaCPP(llm, prompt=prompt),
}

# 🗨️ **BERTopic**

By pre-calculating the embeddings for each document, we can speed-up additional exploration steps and use the embeddings to quickly iterate over BERTopic's hyperparameters if needed.

🔥 **TIP**: You can find a great overview of good embeddings for clustering on the [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [14]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.cluster import HDBSCAN
# from umap import UMAP
# from hdbscan import HDBSCAN

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")

Load pretrained SentenceTransformer: BAAI/bge-small-en
Use pytorch device: cuda


In [15]:
%%time
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/3211 [00:00<?, ?it/s]

CPU times: user 2min 54s, sys: 15.5 s, total: 3min 10s
Wall time: 1min 23s


In [16]:
%%time

# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

/home/kuba/.cache/pypoetry/virtualenvs/github-search-hM2r__Rf-py3.10/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


CPU times: user 9min 39s, sys: 6min 24s, total: 16min 4s
Wall time: 1min 19s


In [17]:
# Define sub-models
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom')#, prediction_data=True)

# 🔥 **Training**

Now that we have our models prepared, we can start training our topic model! We supply BERTopic with the sub-models of interest, run `.fit_transform`, and see what kind of topics we get.

In [18]:
from bertopic import BERTopic

topic_model = BERTopic(
  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  representation_model=representation_model,
  # Hyperparameters
  top_n_words=10,
  verbose=True
)

In [19]:
%%time
# Train model

topics, probs = topic_model.fit_transform(docs.to_list(), embeddings)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/kuba/.cache/pypoetry/virtualenvs/github-search-hM2r__Rf-py3.10/lib/python3.10/site-package │
│ s/IPython/core/magics/execution.py:1340 in time                                                  │
│                                                                                                  │
│   1337 │   │   else:                                                                             │
│   1338 │   │   │   st = clock2()                                                                 │
│   1339 │   │   │   try:                                                                          │
│ ❱ 1340 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1341 │   │   │   │   out=None                                                                  │
│   1342 │   │   │   │   # multi-line %%time case                                                  │
│   1343 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:3                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'list' object has no attribute 'to_list'

Now that we are done training our model, let's see what topics were generated:

In [20]:
# Show topics
topic_model.get_topic_info()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Show topics                                                                                │
│ ❱ 2 topic_model.get_topic_info()                                                                 │
│   3                                                                                              │
│                                                                                                  │
│ /home/kuba/.cache/pypoetry/virtualenvs/github-search-hM2r__Rf-py3.10/lib/python3.10/site-package │
│ s/bertopic/_bertopic.py:1505 in get_topic_info                                                   │
│                                                                                                  │
│   1502 │   │   info_df = topic_model.get_topic_info()                                            │
│   1503 │   │   ```                                                                               │
│   1504 │   │   """                                                                               │
│ ❱ 1505 │   │   check_is_fitted(self)                                                             │
│   1506 │   │                                                                                     │
│   1507 │   │   info = pd.DataFrame(self.topic_sizes_.items(), columns=["Topic", "Count"]).sort_  │
│   1508 │   │   info["Name"] = info.Topic.map(self.topic_labels_)                                 │
│                                                                                                  │
│ /home/kuba/.cache/pypoetry/virtualenvs/github-search-hM2r__Rf-py3.10/lib/python3.10/site-package │
│ s/bertopic/_utils.py:76 in check_is_fitted                                                       │
│                                                                                                  │
│    73 │   │      "appropriate arguments before using this estimator.")                           │
│    74 │                                                                                          │
│    75 │   if topic_model.topics_ is None:                                                        │
│ ❱  76 │   │   raise ValueError(msg % {'name': type(topic_model).__name__})                       │
│    77                                                                                            │
│    78                                                                                            │
│    79 class NotInstalled:                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this 
estimator.

# 📊 Visualization with DataMapPlot

In [ ]:
import PIL
import numpy as np
import requests

We can use the labels created by the LLM and assign them to topics that we have created:

In [ ]:
import datamapplot
import re

# Create a label for each document
llm_labels = [re.sub(r'\W+', ' ', label[0][0].split("\n")[0].replace('"', '')) for label in topic_model.get_topics(full=True)["LLM"].values()]
llm_labels = [label if label else "Unlabelled" for label in llm_labels]
all_labels = [llm_labels[topic+topic_model._outliers] if topic != -1 else "Unlabelled" for topic in topics]

# Run the visualization
datamapplot.create_plot(
    reduced_embeddings,
    all_labels,
    label_font_size=11,
    title="ArXiv - BERTopic",
    sub_title="Topics labeled with `openhermes-2.5-mistral-7b`",
    label_wrap_width=20,
    use_medoids=True,
    logo=bertopic_logo,
    logo_width=0.16
)